In [ ]:
import runcode
runcode.verbosity = 0

The runcode module has two functions, def_code (which saves the symbols you define) and run_code (which does not).

In [ ]:
# define a flag in an environment variable
dem = "-DM=1"

In [ ]:
%%run_code {dem} -DV=2 
std::cout << "hello M=" << M << " V=" << V << "\n";

In [ ]:
%%def_code
int fib(int n) {
    if(n < 2)
        return n;
    return fib(n-1)+fib(n-2);
}

The code below takes a while to run.

In [ ]:
%%run_code
std::cout << fib(42) << std::endl;

In [ ]:
%%def_code
int n42=fib(42);

This code also takes a while to run, unfortunately.

In [ ]:
%%run_code
std::cout << "n42=" << n42 << std::endl;

Even though the symbol n42 is still defined, this code is fast. Apparently, lazy optimization works well.

In [ ]:
%%run_code
std::cout << "n10=" << fib(10) << std::endl;

We can, potentially, fix this problem by using constexpr

In [ ]:
%%def_code
constexpr int fibc(int n) {
    if(n < 2) {
        return n;
    } else {
        return fibc(n-1)+fibc(n-2);
    }
}

In [ ]:
cexpr_flags = "-fconstexpr-steps=5000000 -fconstexpr-depth=40"

Unfortunately, setting of constexpr inside a module doesn't work at the moment

In [ ]:
%%def_code {cexpr_flags}
extern int fc28;
%%lib
constexpr int fc28_ = fibc(28);
int fc28 = fc28_;

This retrieves the value quickly...

In [ ]:
%%run_code

std::cout << "fc28=" << fc28 << std::endl;

Thee's another way to store values between cell evaluations

In [ ]:
%%def_code
struct Counter {
    int n;
    Counter() : n(0) {}
    ~Counter() { std::cout << "reset counter"; }
    void count() {
        std::cout << "n=" << (n++) << std::endl;
    }
};

In [ ]:
%%run_code
Seg seg("mem");
Counter *c = seg.allocate<Counter>("counter");
c->count();
if(c->n == 5)
   seg.remove(c);

In [ ]:
%%run_code
Seg seg("mem");
Array<double>& arr=*seg.allocate_array<double>("data", 100);
if(arr.init()) std::cout << "init" << std::endl;
seg.remove(&arr);

In [ ]:
%%run_code
#include <math.h>

Seg seg("mem");
const int N=100;
Array<double>& a = *seg.allocate_array<double>("data",N);
Array<double>& b = *seg.allocate_array<double>("data2",N);
double dx = 1.0/a.size();
for(int i=0;i<a.size();i++) {
    double x = i*dx;
    a[i] = x;
    b[i] = sin(x);
}

In [ ]:
import clangw
import numpy as np
import matplotlib.pyplot as plt

abuf = clangw.allocate_array("mem","data",100)
a = np.asarray(abuf)

bbuf = clangw.allocate_array("mem","data2",100)
b = np.asarray(bbuf)

plt.plot(a,b)

Parallel execution is straighforward

In [ ]:
%%run_code
import <iostream>;
import <future>;
auto a = std::async(std::launch::async, [](){ return 42; });
std::cout << "a=" << a.get() << std::endl;

In [ ]:
import re
import os

def rmitem(item_pattern,items):
    new_items = []
    for item in items:
        if not re.match(item_pattern, item):
            new_items += [item]
    return new_items

os.environ["LD_LIBRARY_PATH"]="/usr/local/lib64"
os.environ["PKG_CONFIG_PATH"]="/usr/local/lib64/pkgconfig"
runcode.verbose = False
from subprocess import Popen, PIPE
p = Popen("pkg-config --cflags --libs hpx_application_release".split(" "),
          stdout=PIPE,stderr=PIPE,universal_newlines=True)
out, err = p.communicate()
# Set the application flags
runcode.appflags = rmitem(r'-std=.*',out.strip().split(' '))

p = Popen("pkg-config --cflags hpx_application_release".split(" "),
          stdout=PIPE,stderr=PIPE,universal_newlines=True)
out, err = p.communicate()
# Set the module flags
runcode.modflags = rmitem(r'-std=.*',out.strip().split(' '))

In [ ]:
%%run_code
#include <hpx/hpx.hpp>
#include <hpx/hpx_main.hpp>

auto a = hpx::async([](){ return 42; });
std::cout << a.get() << std::endl;

Running HPX requires that we set up some stuff...

For some reason I don't understand, no %%def_code segment can include hpx.